x = 127.059738
y = 37.483606
new_store = {'store_nm' : '새점포', 'sale_grade' : None, 'sale_grade_ahi' : None, 'sale_grade_hist' : None}

주차장 (parking_200m, 500m, 1000m)
for radius in [200, 500, 1000]:
category_group_code = "PK6"
result = kakao.search_category(category_group_code, x, y, radius)
result # total_count 확인
new_store[f'parking_{radius}m'] = len(result)

학원
for radius in [200, 500, 1000]:
category_group_code = 'AC5'
result = kakao.search_category(category_group_code, x, y, radius)
result # total_count 확인
new_store[f'academy_{radius}m'] = len(result)

학교
for radius in [500]:
category_group_code = 'SC4'
result = kakao.search_category(category_group_code, x, y, radius)
result # total_count 확인
new_store[f'school_{radius}m'] = len(result)

# 차주에 와서 다듬겠습니다.

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import numpy as np
import pymysql
from haversine import haversine

# 해당 영역에 속하지 않는 좌표 : 127.04648 37.49589
# 해당 영역에 속하는 좌표 : 127.059738 37.483606

# 변수 연결
- 아파트 결측치 확인 : 126.85843 37.5578

In [11]:
x,y = map(float, input().split())
url = {'url': 'https://raw.githubusercontent.com/mjs1995/Visualization/main/dash_app/data/k7store.png', 'width': 256, 'height': 256, 'anchorY': 256}
mart_input = pd.DataFrame({'x': [x], 'y':[y], 'icon_url': [url]})
mart_input

126.85843 37.55781


,x,y,icon_url
0,126.85843,37.55781,{'url': 'https://raw.githubusercontent.com/mjs...


# 상권영역 연결 TRDAR_SE_C -> TBGIS.geojson 에러로 csv형태로 수정
## trdar_se_a / trdar_se_r / trdar_se_u / trdar_se_d

In [12]:
# 데이터 불러오는 함수
def import_db(data):
    db = pymysql.connect(
        user='bigdata', 
        passwd='ldccBigdata!23', 
        host='10.231.238.14', 
        db='trade_area_analysis', 
        charset='utf8'
        )
    cursor = db.cursor(pymysql.cursors.DictCursor)
    sql = "SELECT * FROM " +data+";"
    cursor.execute(sql)
    df = cursor.fetchall()
    df = pd.DataFrame(df)
    return df


def polygon_to_coordinates(x):
    lon, lat = x.exterior.xy
    return [[x, y] for x, y in zip(lon, lat)]

def multipolygon_to_coordinates(x): 
    lon, lat = x[0].exterior.xy
    return [[x, y] for x, y in zip(lon, lat)]

def center_point(x):
    first = np.mean([w[0] for w in x])
    second = np.mean([w[1] for w in x])
    return [first, second]

def lat_lon(x):
    first = [w[0] for w in x]
    second = [w[1] for w in x]
    return np.array([first,second])

In [13]:
def trdar_merge(mart_input):
    global lat_lon
    # 경계데이터인 geojson 파일 불러옴 
    df = gpd.read_file('TBGIS.geojson')
    df['coordinates'] = df['geometry'].apply(polygon_to_coordinates)
    df['center'] = df['coordinates'].apply(center_point)
    df['lat_lon'] = df['coordinates'].apply(lat_lon)
    
    # 경계데이터의 경계좌표값 지정
    lat_lon = df['lat_lon'].values
    min_1 = np.array([min(w[0]) for w in lat_lon]) 
    max_1 = np.array([max(w[0]) for w in lat_lon])  
    min_2 = np.array([min(w[1]) for w in lat_lon]) 
    max_2 = np.array([max(w[1]) for w in lat_lon]) 

    # 사용자 데이터 xy좌표값과 index값 지정
    mart_input['xy'] = [np.array([i,j]) for i,j in zip(mart_input['x'],mart_input['y'])]
    mart_input['idx'] = 0
    geoX = [w[0] for w in mart_input['xy']]
    geoy = [w[1] for w in mart_input['xy']]

    # input데이터가 경계데이터 안에 속하면 해당 index를 추출 그렇지 않으면 -99표시 
    for i in range(len(geoX)):
        try:
            mart_input.at[i,'idx'] = np.where(( min_1<=geoX[i]) & (max_1>=geoX[i]) & (min_2<=geoy[i]) & (max_2>=geoy[i]))[0][0]      
        except IndexError:
            mart_input.at[i,'idx'] = -99    
    
    # input데이터가 경계데이터 안에 속하지 않으면 haversine을 이용하여 최단거리 계산, dis는 haversine값을 표기하기 위해 생성 
    test = mart_input[mart_input.idx == -99]
    df['dis'] = 0
    df.dis = df.dis.astype(float)

    for i in range(len(df.center)):
        df.at[i,'dis'] = float(haversine(df.center.iloc[i],test.iloc[0].xy))
    
    # haversine값이 제일 낮은 값의 index를 input데이터에 삽입
    mart_input.at[0,'idx'] = df[df.dis == sorted(df.dis)[0]].index[0]
    mart_input = mart_input[mart_input.index == 0]
    
    # input데이터와 상권영역 데이터 merge
    df.reset_index(inplace=True)
    df.rename(columns  = {'index': 'idx'}, inplace = True)
    res = pd.merge(mart_input, df, how='inner', on='idx')
    return res

In [14]:
res = trdar_merge(mart_input)

In [15]:
res

,x,y,icon_url,xy,idx,TRDAR_SE_C,TRDAR_SE_1,TRDAR_CD,TRDAR_CD_N,XCNTS_VALU,YDNTS_VALU,SIGNGU_CD,ADSTRD_CD,STDR_YM_CD,geometry,coordinates,center,lat_lon,dis
0,126.85843,37.55781,{'url': 'https://raw.githubusercontent.com/mjs...,"[126.85843, 37.55781]",61,R,전통시장,1001407,등마루시장,187644,450807,11500,11500520,201810,"POLYGON ((126.85996 37.55703, 126.86017 37.556...","[[126.85996224811606, 37.55702845091507], [126...","[126.86008533289598, 37.5567957918485]","[[126.85996224811606, 126.86016615629869, 126....",0.196102


## 더미변수 생성

In [16]:
def trdar_dummies(res):
    a = pd.get_dummies(res.TRDAR_SE_1)
    res['trdar_se_a'] = 0
    res['trdar_se_r'] = 0
    res['trdar_se_d'] = 0
    res['trdar_se_u'] = 0
    
    if a.columns[0] == '골목상권':
        res['trdar_se_a'] += 1
    elif a.columns[0] == '전통시장':
        res['trdar_se_r'] += 1
    elif a.columns[0] == '발달상권':
        res['trdar_se_d'] += 1
    else:
        res['trdar_se_u'] += 1
    return res

In [17]:
trdar_dummies(res)

,x,y,icon_url,xy,idx,TRDAR_SE_C,TRDAR_SE_1,TRDAR_CD,TRDAR_CD_N,XCNTS_VALU,...,STDR_YM_CD,geometry,coordinates,center,lat_lon,dis,trdar_se_a,trdar_se_r,trdar_se_d,trdar_se_u
0,126.85843,37.55781,{'url': 'https://raw.githubusercontent.com/mjs...,"[126.85843, 37.55781]",61,R,전통시장,1001407,등마루시장,187644,...,201810,"POLYGON ((126.85996 37.55703, 126.86017 37.556...","[[126.85996224811606, 37.55702845091507], [126...","[126.86008533289598, 37.5567957918485]","[[126.85996224811606, 126.86016615629869, 126....",0.196102,0,1,0,0


# apt_avg_ae / apt_avg_price

## 아파트 데이터가 nan이 아닐때

In [19]:
res.rename(columns  = {'TRDAR_CD': 'trdar_cd'}, inplace = True)
res.trdar_cd = res.trdar_cd.astype('int64')

apartment = import_db('apartment')
apartment1 = apartment[apartment.stdr_yy_cd==2021]
apartment1 = apartment1.drop_duplicates(subset = "trdar_cd")
apart_res = pd.merge(res, apartment1, how='left', on='trdar_cd')

## 아파트 가격과 아파트 면적이 없는경우 최단거리 이용 -> 해당 데이터에 x,y 또는 주소값 존재x

In [30]:
apart_res

,x,y,icon_url,xy,idx,TRDAR_SE_C,TRDAR_SE_1,trdar_cd,TRDAR_CD_N,XCNTS_VALU,...,apt_co,price_1_less,price_1_above,price_2_above,price_3_above,price_4_above,price_5_above,price_6_above,avg_ae,avg_price
0,126.85843,37.55781,{'url': 'https://raw.githubusercontent.com/mjs...,"[126.85843, 37.55781]",61,R,전통시장,1001407,등마루시장,187644,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
apartment

,stdr_yy_cd,stdr_qu_cd,trdar_se_cd,trdar_se_cd_nm,trdar_cd,trdar_cd_nm,apt_co,price_1_less,price_1_above,price_2_above,price_3_above,price_4_above,price_5_above,price_6_above,avg_ae,avg_price
0,2021,1,U,관광특구,1001495,잠실 관광특구,22,16.0,348.0,283.0,83.0,70.0,16.0,1156.0,89,527347107
1,2021,1,U,관광특구,1001494,종로?청계 관광특구,5,77.0,279.0,NaN,NaN,NaN,NaN,NaN,39,112097903
2,2021,1,U,관광특구,1001493,동대문패션타운 관광특구,8,NaN,472.0,159.0,22.0,NaN,24.0,271.0,53,237967427
3,2021,1,U,관광특구,1001491,이태원 관광특구,71,14.0,87.0,98.0,221.0,123.0,91.0,87.0,82,409473123
4,2021,1,R,전통시장,1001490,길동복조리시장,37,24.0,322.0,144.0,164.0,161.0,NaN,1.0,62,183067389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40084,2021,1,A,골목상권,1000005,백석동길,41,19.0,112.0,101.0,34.0,16.0,1.0,NaN,84,214023214
40085,2021,1,A,골목상권,1000004,명륜길,41,147.0,107.0,20.0,2.0,NaN,NaN,NaN,51,105457249
40086,2021,1,A,골목상권,1000003,돈화문로11가길,5,21.0,131.0,243.0,35.0,NaN,NaN,NaN,35,161118305
40087,2021,1,A,골목상권,1000002,난계로27길,5,253.0,17.0,NaN,NaN,NaN,NaN,NaN,15,87700490


In [27]:
apart_res.columns

Index(['x', 'y', 'icon_url', 'xy', 'idx', 'TRDAR_SE_C', 'TRDAR_SE_1',
       'trdar_cd', 'TRDAR_CD_N', 'XCNTS_VALU', 'YDNTS_VALU', 'SIGNGU_CD',
       'ADSTRD_CD', 'STDR_YM_CD', 'geometry', 'coordinates', 'center',
       'lat_lon', 'dis', 'trdar_se_a', 'trdar_se_r', 'trdar_se_d',
       'trdar_se_u', 'stdr_yy_cd', 'stdr_qu_cd', 'trdar_se_cd',
       'trdar_se_cd_nm', 'trdar_cd_nm', 'apt_co', 'price_1_less',
       'price_1_above', 'price_2_above', 'price_3_above', 'price_4_above',
       'price_5_above', 'price_6_above', 'avg_ae', 'avg_price'],
      dtype='object')

In [28]:
apart_res.lat_lon

0    [[126.85996224811606, 126.86016615629869, 126....
Name: lat_lon, dtype: object

In [23]:
apartment1

,stdr_yy_cd,stdr_qu_cd,trdar_se_cd,trdar_se_cd_nm,trdar_cd,trdar_cd_nm,apt_co,price_1_less,price_1_above,price_2_above,price_3_above,price_4_above,price_5_above,price_6_above,avg_ae,avg_price
0,2021,1,U,관광특구,1001495,잠실 관광특구,22,16.0,348.0,283.0,83.0,70.0,16.0,1156.0,89,527347107
1,2021,1,U,관광특구,1001494,종로?청계 관광특구,5,77.0,279.0,NaN,NaN,NaN,NaN,NaN,39,112097903
2,2021,1,U,관광특구,1001493,동대문패션타운 관광특구,8,NaN,472.0,159.0,22.0,NaN,24.0,271.0,53,237967427
3,2021,1,U,관광특구,1001491,이태원 관광특구,71,14.0,87.0,98.0,221.0,123.0,91.0,87.0,82,409473123
4,2021,1,R,전통시장,1001490,길동복조리시장,37,24.0,322.0,144.0,164.0,161.0,NaN,1.0,62,183067389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1338,2021,1,A,골목상권,1000005,백석동길,41,19.0,112.0,101.0,34.0,16.0,1.0,NaN,84,214023214
1339,2021,1,A,골목상권,1000004,명륜길,41,147.0,107.0,20.0,2.0,NaN,NaN,NaN,51,105457249
1340,2021,1,A,골목상권,1000003,돈화문로11가길,5,21.0,131.0,243.0,35.0,NaN,NaN,NaN,35,161118305
1341,2021,1,A,골목상권,1000002,난계로27길,5,253.0,17.0,NaN,NaN,NaN,NaN,NaN,15,87700490


In [29]:
res.columns

Index(['x', 'y', 'icon_url', 'xy', 'idx', 'TRDAR_SE_C', 'TRDAR_SE_1',
       'trdar_cd', 'TRDAR_CD_N', 'XCNTS_VALU', 'YDNTS_VALU', 'SIGNGU_CD',
       'ADSTRD_CD', 'STDR_YM_CD', 'geometry', 'coordinates', 'center',
       'lat_lon', 'dis', 'trdar_se_a', 'trdar_se_r', 'trdar_se_d',
       'trdar_se_u'],
      dtype='object')

In [26]:
df = gpd.read_file('TBGIS.geojson')
df['coordinates'] = df['geometry'].apply(polygon_to_coordinates)
df['center'] = df['coordinates'].apply(center_point)
df

,TRDAR_SE_C,TRDAR_SE_1,TRDAR_CD,TRDAR_CD_N,XCNTS_VALU,YDNTS_VALU,SIGNGU_CD,ADSTRD_CD,STDR_YM_CD,geometry,coordinates,center
0,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,201810,"POLYGON ((126.95719 37.47880, 126.95689 37.478...","[[126.95719365538379, 37.47880015198919], [126...","[126.95640327261074, 37.478103026584876]"
1,R,전통시장,1001454,봉천제일종합시장,195147,442413,11620,11620595,201810,"POLYGON ((126.94472 37.48094, 126.94472 37.480...","[[126.94472306184022, 37.48094217073382], [126...","[126.94483804016875, 37.48114759819052]"
2,R,전통시장,1001474,도곡시장,204551,444227,11680,11680650,201810,"POLYGON ((127.05253 37.49702, 127.05208 37.496...","[[127.0525281501012, 37.49701605081407], [127....","[127.05152334248184, 37.49752673943796]"
3,R,전통시장,1001475,강남개포시장,206065,443310,11680,11680670,201810,"POLYGON ((127.06765 37.48935, 127.06904 37.489...","[[127.06765349267593, 37.48935051845855], [127...","[127.06841190581231, 37.48917027745501]"
4,R,전통시장,1001412,화곡본동시장,186203,449328,11500,11500590,201810,"POLYGON ((126.84379 37.54250, 126.84335 37.542...","[[126.8437914682621, 37.54250390407221], [126....","[126.84384355075886, 37.54332986597518]"
...,...,...,...,...,...,...,...,...,...,...,...,...
1491,D,발달상권,1001044,양재역_3,202755,443008,11650,11650520,201810,"POLYGON ((127.03268 37.48735, 127.03196 37.487...","[[127.03268315537306, 37.48735107492279], [127...","[127.03139922324311, 37.48657322496386]"
1492,D,발달상권,1001030,양재역_2,203216,442691,11650,11650651,201810,"POLYGON ((127.03805 37.48366, 127.03748 37.483...","[[127.03804783879356, 37.4836621827563], [127....","[127.03656512262806, 37.48389557268978]"
1493,D,발달상권,1001031,서울 관악구 신림역_1,193706,442615,11620,11620645,201810,"POLYGON ((126.92889 37.48178, 126.92910 37.481...","[[126.92889182512647, 37.48177812909257], [126...","[126.92858850015666, 37.48287184526357]"
1494,D,발달상권,1001032,서울 관악구 신림역_2,193880,442602,11620,11620645,201810,"POLYGON ((126.93000 37.48128, 126.92990 37.482...","[[126.93000097740867, 37.481279632550375], [12...","[126.93085968887036, 37.48254420850662]"


# total_worker -> apart와 같이 예외적인 상황 고려,,

In [34]:
worker = import_db('worker_population')

In [36]:
# 이경우 문제,,
worker[worker.trdar_cd_nm.str.contains('등촌')]

,stdr_yy_cd,stdr_qu_cd,trdar_se_cd,trdar_se_cd_nm,trdar_cd,trdar_cd_nm,total_worker,male_worker,female_worker
304,2021,1,D,발달상권,1001174,등촌역,64,37,27
887,2021,1,A,골목상권,1000588,등촌로5길,1524,514,1010
888,2021,1,A,골목상권,1000587,등촌로55길,432,323,109
889,2021,1,A,골목상권,1000586,등촌로51길,239,137,102
890,2021,1,A,골목상권,1000585,등촌로39길,313,146,167
891,2021,1,A,골목상권,1000584,등촌로13길,183,128,55


In [38]:
res= pd.merge(res, worker, how='left', on='trdar_cd')

In [39]:
res 

,x,y,icon_url,xy,idx,TRDAR_SE_C,TRDAR_SE_1,trdar_cd,TRDAR_CD_N,XCNTS_VALU,...,trdar_se_d,trdar_se_u,stdr_yy_cd,stdr_qu_cd,trdar_se_cd,trdar_se_cd_nm,trdar_cd_nm,total_worker,male_worker,female_worker
0,126.85843,37.55781,{'url': 'https://raw.githubusercontent.com/mjs...,"[126.85843, 37.55781]",61,R,전통시장,1001407,등마루시장,187644,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# tot_flpop_co -> 예외 없어서 그냥 merge만 하면됨

In [44]:
df_fl = import_db('flpop_co')
df_fl1 = df_fl[df_fl.stdr_yy_cd == 2020]
df_fl1 = df_fl1[df_fl1.stdr_qu_cd ==4]
df_fl1.head(2)

,stdr_yy_cd,stdr_qu_cd,trdar_se_cd,trdar_se_cd_nm,trdar_cd,trdar_cd_nm,tot_flpop_co,ml_flpop_co,fml_flpop_co
0,2020,4,U,관광특구,1001496,강남 마이스 관광특구,90836,46313,44523
1,2020,4,U,관광특구,1001495,잠실 관광특구,2956754,1445027,1511727


In [45]:
res0 = pd.merge(res, df_fl1, how='inner', on='trdar_cd')

In [46]:
res0

,x,y,icon_url,xy,idx,TRDAR_SE_C,TRDAR_SE_1,trdar_cd,TRDAR_CD_N,XCNTS_VALU,...,male_worker,female_worker,stdr_yy_cd_y,stdr_qu_cd_y,trdar_se_cd_y,trdar_se_cd_nm_y,trdar_cd_nm_y,tot_flpop_co,ml_flpop_co,fml_flpop_co
0,126.85843,37.55781,{'url': 'https://raw.githubusercontent.com/mjs...,"[126.85843, 37.55781]",61,R,전통시장,1001407,등마루시장,187644,...,NaN,NaN,2020,4,R,전통시장,등마루시장,38013,17306,20706


# tot_repop_co -> 단순 merge

In [50]:
repop = import_db('repop_co')
repop1 = repop[repop.stdr_yy_cd == 2021]
res_repop = pd.merge(res0, repop1, how='inner', on='trdar_cd')
res_repop

,x,y,icon_url,xy,idx,TRDAR_SE_C,TRDAR_SE_1,trdar_cd,TRDAR_CD_N,XCNTS_VALU,...,trdar_se_cd_nm,tot_repop_co,ml_repop_co,fml_repop_co,tot_hshld_co,apt_hshld_co,non_apt_hshld_co,trdar_cd_nm,Column1,Column2
0,126.85843,37.55781,{'url': 'https://raw.githubusercontent.com/mjs...,"[126.85843, 37.55781]",61,R,전통시장,1001407,등마루시장,187644,...,전통시장,3,2,1,3,0,3,등마루시장,None,None


# cnsmr_popltn_co

> 이부분 코드가 없어져서 확인해서 정리하겠습니다

In [64]:
import requests
import json

In [65]:
 class KakaoLocalAPI:
    """
    Kakao Local API 컨트롤러
    """

    def __init__(self, rest_api_key):
        """
        Rest API키 초기화 및 기능 별 URL 설정
        """

        # REST API 키 설정
        self.rest_api_key = rest_api_key
        self.headers = {"Authorization": "KakaoAK {}".format(rest_api_key)}

        # 서비스 별 URL 설정

        # 01 주소 검색
        self.URL_01 = "https://dapi.kakao.com/v2/local/search/address.json"
        # 02 좌표-행정구역정보 변환
        self.URL_02 = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json"
        # 03 좌표-주소 변환
        self.URL_03 = "https://dapi.kakao.com/v2/local/geo/coord2address.json"
        # 04 좌표계 변환
        self.URL_04 = "https://dapi.kakao.com/v2/local/geo/transcoord.json"
        # 05 키워드 검색
        self.URL_05 = "https://dapi.kakao.com/v2/local/search/keyword.json"
        # 06 카테고리 검색
        self.URL_06 = "https://dapi.kakao.com/v2/local/search/category.json"
        
    def search_address(self, query, analyze_type=None, page=None, size=None):
        """
        01 주소 검색
        """
        params = {"query": f"{query}"}

        if analyze_type != None:
            params["analyze_type"] = f"{analyze_type}"

        if page != None:
            params['page'] = f"{page}"

        if size != None:
            params['size'] = f"{size}"

        res = requests.get(self.URL_01, headers=self.headers, params=params)
        document = json.loads(res.text)

        return document
    
    def geo_coord2regioncode(self, x, y, input_coord=None, output_coord=None):
        """
        02 좌표-행정구역정보 변환
        """
        params = {"x": f"{x}",
                  "y": f"{y}"}
        
        if input_coord != None:
            params['input_coord'] = f"{input_coord}"
        
        if output_coord != None:
            params['output_coord'] = f"{output_coord}"
            
        res = requests.get(self.URL_02, headers=self.headers, params=params)
        document = json.loads(res.text)
        
        return document
    
    def geo_coord2address(self, x, y, input_coord=None):
        """
        03 좌표-주소 변환
        """
        params = {"x": f"{x}",
                  "y": f"{y}"}
        
        if input_coord != None:
            params['input_coord'] = f"{input_coord}"
            
        res = requests.get(self.URL_03, headers=self.headers, params=params)
        document = json.loads(res.text)
        
        return document
    
    def geo_transcoord(self, x, y, output_coord, input_coord=None):
        """
        04 좌표계 변환
        """
        params = {"x": f"{x}",
                  "y": f"{y}",
                  "output_coord": f"{output_coord}"}
        
        if input_coord != None:
            params['input_coord'] = f"{input_coord}"
        
        res = requests.get(self.URL_04, headers=self.headers, params=params)
        document = json.loads(res.text)
        
        return document
    
    def search_keyword(self,query,category_group_code=None,x=None,y=None,radius=None,rect=None,page=None,size=None,sort=None):
        """
        05 키워드 검색
        """
        params = {"query": f"{query}"}
        
        if category_group_code != None:
            params['category_group_code'] = f"{category_group_code}"
        if x != None:
            params['x'] = f"{x}"
        if y != None:
            params['y'] = f"{y}"
        if radius != None:
            params['radius'] = f"{radius}"
        if rect != None:
            params['rect'] = f"{rect}"
        if page != None:
            params['page'] = f"{page}"
        if size != None:
            params['size'] = f"{size}"
        if sort != None:
            params['sort'] = f"{sort}"
        
        res = requests.get(self.URL_05, headers=self.headers, params=params)
        document = json.loads(res.text)
        
        return document
    
    def search_category(self, category_group_code, x, y, radius=None, rect=None, page=None, size=None, sort=None):
        """
        06 카테고리 검색
        """
        params = {'category_group_code': f"{category_group_code}",
                  'x': f"{x}",
                  'y': f"{y}"}
        
        if radius != None:
            params['radius'] = f"{radius}"
        if rect != None:
            params['rect'] = f"{rect}"
        if page != None:
            params['page'] = f"{page}"
        if size != None:
            params['size'] = f"{size}"
        if sort != None:
            params['sort'] = f"{sort}"
            
        res = requests.get(self.URL_06, headers=self.headers, params=params)
        document = json.loads(res.text)
        
        return document

In [66]:
rest_api_key = "7c9ba66d98466d82c0654496eabbc545"

kakao = KakaoLocalAPI(rest_api_key)

In [67]:
for i,(x,y) in enumerate(zip(res['x'],res['y'])):
    input_coord = "WGS84" 
    output_coord = "TM" 
    result_2 = kakao.geo_coord2regioncode(x,y, input_coord, output_coord)
    res.at[i,'adstrd_nm'] = result_2['documents'][1]['region_3depth_name']


#df_conv_cp.de = df_conv_cp.de.astype(str)
#df_conv_cp['month'] = df_conv_cp.de.str[4:6]
#df_conv_cp1 = df_conv_cp[df_conv_cp.month == str(10)]
#df_conv_cp2 = df_conv_cp1.groupby(['gov_dn_cd','adstrd_nm']).sum()
#df_conv_cp2.reset_index(inplace=True)
#b = df_conv_cp2[df_conv_cp2.gov_dn_cd !=1162068500]
#data = pd.merge(df, b, how='inner', on='adstrd_nm')

In [68]:
res

,x,y,icon_url,xy,idx,TRDAR_SE_C,TRDAR_SE_1,trdar_cd,TRDAR_CD_N,XCNTS_VALU,...,trdar_se_u,stdr_yy_cd,stdr_qu_cd,trdar_se_cd,trdar_se_cd_nm,trdar_cd_nm,total_worker,male_worker,female_worker,adstrd_nm
0,126.85843,37.55781,{'url': 'https://raw.githubusercontent.com/mjs...,"[126.85843, 37.55781]",61,R,전통시장,1001407,등마루시장,187644,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,등촌1동


In [58]:
df

,TRDAR_SE_C,TRDAR_SE_1,TRDAR_CD,TRDAR_CD_N,XCNTS_VALU,YDNTS_VALU,SIGNGU_CD,ADSTRD_CD,STDR_YM_CD,geometry,coordinates,center
0,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,201810,"POLYGON ((126.95719 37.47880, 126.95689 37.478...","[[126.95719365538379, 37.47880015198919], [126...","[126.95640327261074, 37.478103026584876]"
1,R,전통시장,1001454,봉천제일종합시장,195147,442413,11620,11620595,201810,"POLYGON ((126.94472 37.48094, 126.94472 37.480...","[[126.94472306184022, 37.48094217073382], [126...","[126.94483804016875, 37.48114759819052]"
2,R,전통시장,1001474,도곡시장,204551,444227,11680,11680650,201810,"POLYGON ((127.05253 37.49702, 127.05208 37.496...","[[127.0525281501012, 37.49701605081407], [127....","[127.05152334248184, 37.49752673943796]"
3,R,전통시장,1001475,강남개포시장,206065,443310,11680,11680670,201810,"POLYGON ((127.06765 37.48935, 127.06904 37.489...","[[127.06765349267593, 37.48935051845855], [127...","[127.06841190581231, 37.48917027745501]"
4,R,전통시장,1001412,화곡본동시장,186203,449328,11500,11500590,201810,"POLYGON ((126.84379 37.54250, 126.84335 37.542...","[[126.8437914682621, 37.54250390407221], [126....","[126.84384355075886, 37.54332986597518]"
...,...,...,...,...,...,...,...,...,...,...,...,...
1491,D,발달상권,1001044,양재역_3,202755,443008,11650,11650520,201810,"POLYGON ((127.03268 37.48735, 127.03196 37.487...","[[127.03268315537306, 37.48735107492279], [127...","[127.03139922324311, 37.48657322496386]"
1492,D,발달상권,1001030,양재역_2,203216,442691,11650,11650651,201810,"POLYGON ((127.03805 37.48366, 127.03748 37.483...","[[127.03804783879356, 37.4836621827563], [127....","[127.03656512262806, 37.48389557268978]"
1493,D,발달상권,1001031,서울 관악구 신림역_1,193706,442615,11620,11620645,201810,"POLYGON ((126.92889 37.48178, 126.92910 37.481...","[[126.92889182512647, 37.48177812909257], [126...","[126.92858850015666, 37.48287184526357]"
1494,D,발달상권,1001032,서울 관악구 신림역_2,193880,442602,11620,11620645,201810,"POLYGON ((126.93000 37.48128, 126.92990 37.482...","[[126.93000097740867, 37.481279632550375], [12...","[126.93085968887036, 37.48254420850662]"
